In [1]:
import os
import sys
from lca_algebraic import initProject
import pandas as pd
import numpy as np
import xarray as xr
import lca_algebraic as agb
from lca_algebraic import *
from lca_algebraic.stats import * 
import bw2data, bw2io
import bw2calc
from premise import *
import premise as prm
from premise.geomap import Geomap
from premise_gwp import add_premise_gwp

In [2]:
sys.path.append('/Users/susierwu/Dropbox/DLCA_DynCO2/pdpLCA_GT/') 
from utils import *
from utils.setup import *
from utils.getAct import *
from utils.utils import *
#from utils.switchElec import *

modify the setup.py file for reading Parameter excel sheet. here we use 'data/p_file/p_tech_perSSP_Y.xlsx' as the technology parameter file.  
Define your chosen methods in the setup.py file. Current methods are:  [('IPCC 2021 no LT', 'climate change no LT', 'global warming potential (GWP100) no LT'), ('IPCC 2021', 'climate change', 'GWP 100a, incl. H and bio CO2')]
Mapping premise_remind_DB to SSPx:  {'SSP1-PkBudg500': 'ssp119', 'SSP1-PkBudg1150': 'ssp126', 'SSP2-Base': 'ssp245', 'SSP5-Base': 'ssp585'}
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
pGWP100 are imported as:  [('IPCC 2021 - dpCFsSSP119_MY2030 - year100', 'climate change', 'pGWP100'), ('IPCC 2021 - dpCFsSSP245_MY2030 - year100', 'climate change', 'pGWP100'), ('IPCC 2021 - dpCFsSSP585_MY2030 - year100', 'climate change', 'pGWP100'), ('IPCC 2021 - dpCFsSSP119_MY2040 - year100', 'climate change', 'pGWP100'), ('IPCC 2021 - dpCFsSSP245_MY2040 - year100', 'climate cha

In [3]:
bw2data.projects.set_current('iveo_v1')
mybio = bw2data.Database("biosphere3")
initProject("iveo_v1")

Biosphere database already present!!! No setup is needed


In [6]:
#[m for m in bw2data.methods if 'IPCC 2021' in str(m) and  'dIRF36' in str(m)  ]

### init 

In [4]:
pdb = bw.Database("iveo_Parameterized_v1")
print(len(pdb))

384


In [5]:
#resetParams()
agb.loadParams()
#agb.list_parameters()

[ParamRegistry] Param NMC622_battery_cell_density was already defined in '<project>' : overriding.
[ParamRegistry] Param NCA_battery_cell_density was already defined in '<project>' : overriding.
[ParamRegistry] Param LFP_battery_cell_density was already defined in '<project>' : overriding.
[ParamRegistry] Param LTO_battery_cell_density was already defined in '<project>' : overriding.
[ParamRegistry] Param LSB_battery_cell_density was already defined in '<project>' : overriding.
[ParamRegistry] Param LSB_cell_assmb_NG_MJperkg was already defined in '<project>' : overriding.
[ParamRegistry] Param LSB_cell_assmb_elec_kWhperkg was already defined in '<project>' : overriding.
[ParamRegistry] Param g_truck_vehicle_prod_lightweighting was already defined in '<project>' : overriding.
[ParamRegistry] Param battery_total_capacity_for_gtruck was already defined in '<project>' : overriding.
[ParamRegistry] Param biodiesel_UCO_perc was already defined in '<project>' : overriding.
[ParamRegistry] Pa

## A. prepare all P and UPRs
### 1. define all P

#### vehicle_prod_lightweight_parameter

same lightweight_p as for BEV: 
p_g_truck_lightweighting = newFloatParam(
        name="g_truck_vehicle_prod_lightweighting"

   

## 2. preparing UPRs for ICEV-d g_truck:
1. vehicle production 

#### 2.1 adding ICEV g_truck production: 

inventory: LCI_garbage_truck
https://docs.google.com/spreadsheets/d/1_qotcARay-UyygXMX8Ha16aR90uKgEH81npOohgFgcg/edit#gid=1736639544 

In [6]:
#to delete act:  for act in [act for act in bw2data.Database('iveo_Parameterized_v1') if 'g-truck ICEV-d Vehicle Production' in act['name']]: act.delete()
#to delete all act so no duplication: for act in bw2data.Database('iveo_Parameterized_v1'):  act.delete()

gtruck_final_allsspyear = []

for prm_name in ssp_scn:
    for year_t in ssp_yr: 
        bg_db_ssp_yr = bg_ei_ + prm_name + "_" + str(year_t)            # backgroun database (BD) to use for SSP[x], Year[t]
        copied_prm_UPR_suffix = "_from_" + prm_name + "_" + str(year_t) # copy UPR from BD to PDB for further use/modification if needed
        newUPR_suffix =  "_wh_" + prm_name + "_" + str(year_t)          # naming on any new UPRs created in PDB

        # final one unit of g_truck with lightweighting as Parameter: 
        gtruck_final = agb.newActivity(
                db_name = PDB_NAME,
                name="V1A_g-truck ICEV-d Vehicle Production_" + newUPR_suffix,
                code = "V1A_g-truck_ICEV-d_Vehicle_Production_" + newUPR_suffix,
                unit="one_truck",
                exchanges={
                    agb.findActivity("gearbox, for lorry",  db_name = bg_db_ssp_yr): 161,
                    agb.findActivity('internal combustion engine, for lorry',  db_name = bg_db_ssp_yr): 940,
                    agb.findActivity("transmission, for lorry",   db_name = bg_db_ssp_yr): 233,
                    agb.findActivity('exhaust system, for lorry', db_name = bg_db_ssp_yr): 176,
                    agb.findActivity('other components, for hybrid electric lorry', db_name = bg_db_ssp_yr): 847,
                    agb.findActivity('suspension, for lorry',  db_name = bg_db_ssp_yr): 2527,
                    agb.findActivity('tires and wheels, for lorry', db_name = bg_db_ssp_yr): 1045,
                    agb.findActivity('frame, blanks and saddle, for lorry',  db_name = bg_db_ssp_yr): 4068,
                    agb.findActivity('fuel tank, for diesel vehicle',  db_name = bg_db_ssp_yr): 38.80,
                    agb.findActivity('glider lightweighting',  db_name = bg_db_ssp_yr): g_truck_vehicle_prod_lightweighting,
                    # specific to BEV, many are proxy UPRs from passenger BEV
                    #agb.findActivity('market for power distribution unit, for electric passenger car',  db_name = bg_db_ssp_yr): 4,
                    #agb.findActivity('market for converter, for electric passenger car',   db_name = bg_db_ssp_yr): 25,
                    #agb.findActivity('market for inverter, for electric passenger car',   db_name = bg_db_ssp_yr): 25,
                    #agb.findActivity('other components, for electric lorry',   db_name = bg_db_ssp_yr): 847,
                    #agb.findActivity('market for electric motor, electric passenger car',  db_name = bg_db_ssp_yr): 847,
                    # extra component garbage bin:
                    agb.findActivity('market for reinforcing steel',  db_name = bg_db_ssp_yr): 3404,
                    agb.findActivity('market for aluminium alloy, AlLi',  db_name = bg_db_ssp_yr): 3404,
                    agb.findActivity('market for metal working, average for steel product manufacturing', loc = 'GLO', db_name = bg_db_ssp_yr): 3404,
                    agb.findActivity('market for metal working, average for aluminium product manufacturing', loc = 'GLO', db_name = bg_db_ssp_yr): 3404,
    
                },
            )
        
        for exc in gtruck_final.getExchange(name = "*", single=False): 
            exc['type']='technosphere'
            exc.save()
        gtruck_final.setOutputAmount(1)
        
        #agb.printAct(gtruck_final)
        print(f"end of creating a new UPR for final ICEV-d garbage truck per truck for SSP {prm_name} and year {year_t}, relying on BD {bg_db_ssp_yr}")
        
        #adding to list: 
        gtruck_final_allsspyear.append(gtruck_final)

end of creating a new UPR for final ICEV-d garbage truck per truck for SSP _remind_SSP1-PkBudg500 and year 2030, relying on BD ecoinvent_cutoff_3.9_remind_SSP1-PkBudg500_2030
end of creating a new UPR for final ICEV-d garbage truck per truck for SSP _remind_SSP1-PkBudg500 and year 2040, relying on BD ecoinvent_cutoff_3.9_remind_SSP1-PkBudg500_2040
end of creating a new UPR for final ICEV-d garbage truck per truck for SSP _remind_SSP1-PkBudg500 and year 2050, relying on BD ecoinvent_cutoff_3.9_remind_SSP1-PkBudg500_2050
end of creating a new UPR for final ICEV-d garbage truck per truck for SSP _remind_SSP1-PkBudg1150 and year 2030, relying on BD ecoinvent_cutoff_3.9_remind_SSP1-PkBudg1150_2030
end of creating a new UPR for final ICEV-d garbage truck per truck for SSP _remind_SSP1-PkBudg1150 and year 2040, relying on BD ecoinvent_cutoff_3.9_remind_SSP1-PkBudg1150_2040
end of creating a new UPR for final ICEV-d garbage truck per truck for SSP _remind_SSP1-PkBudg1150 and year 2050, relying

### get already created UPR: 

In [6]:
icevgtruck_finalallsspyear = [] 
for prm_name in ssp_scn:
    for year_t in ssp_yr: 
        bg_db_ssp_yr = bg_ei_ + prm_name + "_" + str(year_t)            # backgroun database (BD) to use for SSP[x], Year[t]
        copied_prm_UPR_suffix = "_from_" + prm_name + "_" + str(year_t) # copy UPR from BD to PDB for further use/modification if needed
        newUPR_suffix =  "_wh_" + prm_name + "_" + str(year_t)          # naming on any new UPRs created in PDB
        # final one unit of g_truck with lightweighting as Parameter: 
        gtruck_final  = agb.findActivity(
                db_name = PDB_NAME,
                name="V1A_g-truck ICEV-d Vehicle Production_" + newUPR_suffix)
        icevgtruck_finalallsspyear.append(gtruck_final)

len(icevgtruck_finalallsspyear)

12

## 2.B. calculate  with static GWP100 vs. pGWP100

#### B.0 test opening Parameter file

In [7]:
root_path = '/Users/susierwu/Dropbox/DLCA_DynCO2/pdpLCA_GT/'
p_file = os.path.abspath(os.path.join(root_path, 'data/P_files/p_tech_perSSP_Y.xlsx'))

for gtruck_final in icevgtruck_finalallsspyear:
    # need a matched YearlyParams for each SSPx:
    str1 = gtruck_final.as_dict()["name"]
    str2 = str1.split("remind_")[1]
    year_to_get_inP = int( str2.split("_")[-1] )
    ssp_remind = str2.split("_")[0] 
    ssp_to_get_inP = ssp_remind_Pname_map.get(ssp_remind)
    
    yearlyParams = loadYearlyParams_multisheet(PARAMS_FILE = p_file, s_name = ['V1A_g_truck'], years = [year_to_get_inP], SSP = ssp_to_get_inP) 
    print(f"for SSP {ssp_to_get_inP}, year {year_to_get_inP}, P:  {yearlyParams}" )

for SSP ssp119, year 2030, P:  {'g_truck_vehicle_prod_lightweighting': [260.0], 'battery_total_capacity_for_gtruck': [376.0]}
for SSP ssp119, year 2040, P:  {'g_truck_vehicle_prod_lightweighting': [314.6], 'battery_total_capacity_for_gtruck': [346.5216]}
for SSP ssp119, year 2050, P:  {'g_truck_vehicle_prod_lightweighting': [380.6660000000001], 'battery_total_capacity_for_gtruck': [319.35430656]}
for SSP ssp126, year 2030, P:  {'g_truck_vehicle_prod_lightweighting': [250.0], 'battery_total_capacity_for_gtruck': [376.0]}
for SSP ssp126, year 2040, P:  {'g_truck_vehicle_prod_lightweighting': [291.6], 'battery_total_capacity_for_gtruck': [353.7784]}
for SSP ssp126, year 2050, P:  {'g_truck_vehicle_prod_lightweighting': [340.1222400000001], 'battery_total_capacity_for_gtruck': [332.8700965599999]}
for SSP ssp245, year 2030, P:  {'g_truck_vehicle_prod_lightweighting': [237.0], 'battery_total_capacity_for_gtruck': [376.0]}
for SSP ssp245, year 2040, P:  {'g_truck_vehicle_prod_lightweighting'

### B.1.0 reading in pGWP100

In [8]:
print(chosen_methods)

[('IPCC 2021 no LT', 'climate change no LT', 'global warming potential (GWP100) no LT'), ('IPCC 2021', 'climate change', 'GWP 100a, incl. H and bio CO2')]


### B.1 read in all parameter under different SSPx in the calc 

In [9]:
#final output to be appended
gtruck_lcia_dict = {}
GWP0_list, GWP1_list, GWP2_list, SSP_list, Year_list = [], [],[],[],[]

for gtruck_final in icevgtruck_finalallsspyear:
    # need a matched YearlyParams for each SSPx:
    str1 = gtruck_final.as_dict()["name"]
    str2 = str1.split("remind_")[1]
    year_to_get_inP = int( str2.split("_")[-1] )
    ssp_remind = str2.split("_")[0] 
    ssp_to_get_inP = ssp_remind_Pname_map.get(ssp_remind)
    
    yearlyParams = loadYearlyParams_multisheet(PARAMS_FILE = p_file, s_name = ['V1A_g_truck'], years = [year_to_get_inP], SSP = ssp_to_get_inP) 
    print(f"for SSP {ssp_to_get_inP}, year {year_to_get_inP}, P:  {yearlyParams}" ) 
    

    """ prepare methods: GWP1_list using static GWP,  GWP2_list using the pGWP100 for the matched v_year """ 
    for pp in pgwp100: 
        if re.findall(r'\d+', ssp_to_get_inP)[0] in str(pp)  and str(year_to_get_inP) in str(pp): 
           gwp_spx_yt = pp 
        
    methods_touse = [chosen_methods[0], chosen_methods[1], gwp_spx_yt] # IPCCGWP100, premiseGWP, pGWP100_per_premiseGWP
    
    df = agb.multiLCAAlgebric(models=[
                        (gtruck_final, 1)], 
                        methods = methods_touse ,
                        **yearlyParams
                    )
                    
    print(f"end of running LCA calc for SSP {ssp_to_get_inP} and year {year_to_get_inP} for {gtruck_final} ")
    display(df)
        
    # prepare for final DF: 
    GWP0 = df.iloc[:,0].values[0]
    GWP1 = df.iloc[:,1].values[0]
    GWP2 = df.iloc[:,2].values[0]
    GWP0_list.append(GWP0)
    GWP1_list.append(GWP1)
    GWP2_list.append(GWP2)
    SSP_list.append(ssp_to_get_inP) 
    Year_list.append(year_to_get_inP)

    # assign df as value to each UPR name (used as key)
    gtruck_lcia_dict[str1] = df

for SSP ssp119, year 2030, P:  {'g_truck_vehicle_prod_lightweighting': [260.0], 'battery_total_capacity_for_gtruck': [376.0]}
end of running LCA calc for SSP ssp119 and year 2030 for 'V1A_g-truck ICEV-d Vehicle Production__wh__remind_SSP1-PkBudg500_2030' (one_truck, GLO, None) 


,climate change no LT - global warming potential (GWP100) no LT[kg CO2-Eq],"climate change - GWP 100a, incl. H and bio CO2[kg CO2-Eq]",climate change - pGWP100[kg CO2-Eq]
V1A_g-truck ICEV-d Vehicle Production__wh__remind_SSP1-PkBudg500_2030,98438.795525,101192.073576,99125.053226


for SSP ssp119, year 2040, P:  {'g_truck_vehicle_prod_lightweighting': [314.6], 'battery_total_capacity_for_gtruck': [346.5216]}
end of running LCA calc for SSP ssp119 and year 2040 for 'V1A_g-truck ICEV-d Vehicle Production__wh__remind_SSP1-PkBudg500_2040' (one_truck, GLO, None) 


,climate change no LT - global warming potential (GWP100) no LT[kg CO2-Eq],"climate change - GWP 100a, incl. H and bio CO2[kg CO2-Eq]",climate change - pGWP100[kg CO2-Eq]
V1A_g-truck ICEV-d Vehicle Production__wh__remind_SSP1-PkBudg500_2040,87588.25938,88860.703452,87556.976391


for SSP ssp119, year 2050, P:  {'g_truck_vehicle_prod_lightweighting': [380.6660000000001], 'battery_total_capacity_for_gtruck': [319.35430656]}
end of running LCA calc for SSP ssp119 and year 2050 for 'V1A_g-truck ICEV-d Vehicle Production__wh__remind_SSP1-PkBudg500_2050' (one_truck, GLO, None) 


...

,climate change no LT - global warming potential (GWP100) no LT[kg CO2-Eq],"climate change - GWP 100a, incl. H and bio CO2[kg CO2-Eq]",climate change - pGWP100[kg CO2-Eq]
V1A_g-truck ICEV-d Vehicle Production__wh__remind_SSP5-Base_2030,119818.908461,122824.628623,119240.283299


for SSP ssp585, year 2040, P:  {'g_truck_vehicle_prod_lightweighting': [261.2925], 'battery_total_capacity_for_gtruck': [376.0]}
end of running LCA calc for SSP ssp585 and year 2040 for 'V1A_g-truck ICEV-d Vehicle Production__wh__remind_SSP5-Base_2040' (one_truck, GLO, None) 


,climate change no LT - global warming potential (GWP100) no LT[kg CO2-Eq],"climate change - GWP 100a, incl. H and bio CO2[kg CO2-Eq]",climate change - pGWP100[kg CO2-Eq]
V1A_g-truck ICEV-d Vehicle Production__wh__remind_SSP5-Base_2040,116338.825106,119487.61151,116288.67107


for SSP ssp585, year 2050, P:  {'g_truck_vehicle_prod_lightweighting': [288.07498125000006], 'battery_total_capacity_for_gtruck': [376.0]}
end of running LCA calc for SSP ssp585 and year 2050 for 'V1A_g-truck ICEV-d Vehicle Production__wh__remind_SSP5-Base_2050' (one_truck, GLO, None) 


,climate change no LT - global warming potential (GWP100) no LT[kg CO2-Eq],"climate change - GWP 100a, incl. H and bio CO2[kg CO2-Eq]",climate change - pGWP100[kg CO2-Eq]
V1A_g-truck ICEV-d Vehicle Production__wh__remind_SSP5-Base_2050,95006.381889,98170.079895,96562.311548


In [10]:
lcia_icevd = pd.DataFrame(zip(SSP_list, Year_list, GWP0_list, GWP1_list, GWP2_list), columns = ["SSP","Year", "IPCC_GWP100", "premise_GWP100", "pGWP100"])
lcia_icevd

,SSP,Year,IPCC_GWP100,premise_GWP100,pGWP100
0,ssp119,2030,98438.795525,101192.073576,99125.053226
1,ssp119,2040,87588.259380,88860.703452,87556.976391
2,ssp119,2050,84479.855746,83778.524593,82663.226564
3,ssp126,2030,109119.170199,112279.012999,110859.387361
4,ssp126,2040,97878.417525,101093.944727,99829.741166
5,ssp126,2050,90469.087132,93527.653790,92371.133833
6,ssp245,2030,114930.409045,118034.668417,115226.970692
7,ssp245,2040,104599.685668,107803.595647,106102.075150
8,ssp245,2050,97562.766626,100837.469129,99868.612059
9,ssp585,2030,119818.908461,122824.628623,119240.283299


#### maximum diff. is only 3% for ICEV : ssp585	v_year[2030]

In [11]:
(lcia_icevd["premise_GWP100"] - lcia_icevd["pGWP100"] )/ lcia_icevd["premise_GWP100"]

0     0.020427
1     0.014672
2     0.013312
3     0.012644
4     0.012505
5     0.012366
6     0.023787
7     0.015784
8     0.009608
9     0.029183
10    0.026772
11    0.016377
dtype: float64

In [12]:
lcia_icevd.to_excel("output/ICEVd_V1A.xlsx",index=False)